# Performance Measures

Model evaluation on test data or by CV, can be done using the evaluation functions available in *PyDTS* and the measures of performance presented in the Methods section.   

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pydts.examples_utils.generate_simulations_data import generate_quick_start_df
import warnings
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')
%matplotlib inline

real_coef_dict = {
    "alpha": {
        1: lambda t: -1 - 0.3 * np.log(t),
        2: lambda t: -1.75 - 0.15 * np.log(t)
    },
    "beta": {
        1: -np.log([0.8, 3, 3, 2.5, 2]),
        2: -np.log([1, 3, 4, 3, 2])
    }
}

n_patients = 50000
n_cov = 5

patients_df = generate_quick_start_df(n_patients=n_patients, n_cov=n_cov, d_times=30, j_events=2, 
                                      pid_col='pid', seed=0, censoring_prob=0.8, 
                                      real_coef_dict=real_coef_dict)

train_df, test_df = train_test_split(patients_df, test_size=0.2)

patients_df.head()

,pid,Z1,Z2,Z3,Z4,Z5,J,T,C,X
0,0,0.548814,0.715189,0.602763,0.544883,0.423655,0,31,10,10
1,1,0.645894,0.437587,0.891773,0.963663,0.383442,0,31,24,24
2,2,0.791725,0.528895,0.568045,0.925597,0.071036,0,17,11,11
3,3,0.087129,0.020218,0.832620,0.778157,0.870012,1,1,31,1
4,4,0.978618,0.799159,0.461479,0.780529,0.118274,0,15,14,14


For example, in the following code, the survival models are estimated based on the two-stage approach and the dataset train_df. Assume that the event of main interest is $j=1$. Then, $\pi_{i1}(t)$ are calculated and stored in pred_df, and finally $\widehat{\mbox{AUC}}_1(t)$, $t=1,\ldots,d$, are provided by

In [2]:
from pydts.fitters import TwoStagesFitter
from pydts.evaluation import *

fitter = TwoStagesFitter()
fitter.fit(df = train_df)
pred_df = fitter.predict_prob_event_j_all(test_df, event=1)
auc_1 = event_specific_auc_at_t_all(pred_df, event=1)
print(f'AUC(t) for event 1 is:')
auc_1

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
AUC(t) for event 1 is:


1     0.988007
2     0.989981
3     0.990528
4     0.992057
5     0.991414
6     0.992463
7     0.992465
8     0.993415
9     0.993273
10    0.992431
11    0.994390
12    0.995089
13    0.992101
14    0.993462
15    0.995626
16    0.994383
17    0.994551
18    0.992995
19    0.993614
20    0.993318
21    0.994786
22    0.992859
23    0.994097
24    0.997663
25    0.998358
26    0.997495
27    0.997700
28    0.998260
29    0.994920
30    0.988650
Name: 1, dtype: float64

Other measures such as $\widehat{\mbox{AUC}}_1$, $\widehat{\mbox{BS}}_1$, $\widehat{\mbox{AUC}}$, and $\widehat{\mbox{BS}}$ can be calculated by

In [3]:
pred_df = fitter.predict_prob_events(test_df)
ibs_1 = event_specific_integrated_brier_score(pred_df, event = 1)
iauc_1 = event_specific_integrated_auc(pred_df, event = 1)
bs = global_brier_score(pred_df)
auc = global_auc(pred_df)

Model evaluation based on K-fold CV and `TwoStagesFitter` can be done by

In [4]:
from pydts.cross_validation import TwoStagesCV

cross_validator = TwoStagesCV()
cross_validator.cross_validate(full_df = patients_df.drop(['C', 'T'], 
                               axis = 1), 
                               n_splits = 5, seed = 0,
                               metrics = ['BS', 'IBS', 'GBS', 
                                          'AUC', 'IAUC', 'GAUC'])

Fitting fold 1/5
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Fitting fold 2/5
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Fitting fold 3/5
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Fitting fold 4/5
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Fitting fold 5/5
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


1         2         3         4         5         6   \
metric fold                                                                 
BS     0    1  0.063049  0.053907  0.045755  0.043523  0.042300  0.046062   
            2  0.024376  0.019168  0.019545  0.017727  0.019594  0.017552   
AUC    0    1  0.652021  0.643642  0.641287  0.643345  0.667766  0.643727   
            2  0.672119  0.677260  0.688408  0.646792  0.686449  0.659598   
BS     1    1  0.063104  0.051763  0.047522  0.045185  0.039390  0.040410   
            2  0.026567  0.020499  0.023158  0.019674  0.017881  0.018550   
AUC    1    1  0.637347  0.653555  0.670154  0.653391  0.674059  0.664124   
            2  0.702004  0.665337  0.698247  0.670620  0.612123  0.662859   
BS     2    1  0.064886  0.049643  0.045128  0.044718  0.040647  0.041574   
            2  0.024451  0.016186  0.021169  0.021732  0.021547  0.021889   
AUC    2    1  0.659157  0.654793  0.649652  0.659449  0.674832  0.660598   
            2  0.657810  0.657075  0.706675  0.713073  0.645342  0.666765   
BS     3    1  0.059994  0.050895  0.045448  0.043562  0.037141  0.043832   
            2  0.024225  0.021932  0.019773  0.015853  0.017388  0.017645   
AUC    3    1  0.659007  0.665953  0.682254  0.636794  0.677254  0.626931   
            2  0.702751  0.677411  0.691677  0.699105  0.657049  0.655707   
BS     4    1  0.064885  0.053155  0.047227  0.045384  0.041575  0.035789   
            2  0.023856  0.019616  0.022393  0.021449  0.020102  0.016353   
AUC    4    1  0.659357  0.636783  0.654152  0.637880  0.631202  0.647121   
            2  0.672312  0.657230  0.687828  0.674431  0.653015  0.616802   

                     7         8         9         10  ...        21  \
metric fold                                            ...             
BS     0    1  0.037669  0.039918  0.038884  0.039600  ...  0.036635   
            2  0.021547  0.020059  0.016670  0.019916  ...  0.018437   
AUC    0    1  0.655765  0.667565  0.570066  0.653903  ...  0.514947   
            2  0.629191  0.626242  0.693669  0.635776  ...  0.612318   
BS     1    1  0.040533  0.037199  0.035189  0.039389  ...  0.048439   
            2  0.012963  0.019118  0.018793  0.020397  ...  0.016248   
AUC    1    1  0.626964  0.639071  0.622290  0.624590  ...  0.470615   
            2  0.732118  0.652722  0.610839  0.666786  ...  0.564700   
BS     2    1  0.037698  0.035049  0.041622  0.038163  ...  0.048869   
            2  0.017873  0.020496  0.015362  0.014820  ...  0.018748   
AUC    2    1  0.594342  0.621234  0.648151  0.634931  ...  0.611217   
            2  0.653272  0.674614  0.565575  0.620555  ...  0.552351   
BS     3    1  0.038703  0.043434  0.035425  0.037136  ...  0.039925   
            2  0.017031  0.018022  0.015532  0.019680  ...  0.022589   
AUC    3    1  0.650321  0.615777  0.639428  0.600239  ...  0.464096   
            2  0.675905  0.646852  0.718233  0.665166  ...  0.594124   
BS     4    1  0.038320  0.040585  0.031461  0.032940  ...  0.038013   
            2  0.019711  0.020459  0.021224  0.017895  ...  0.025120   
AUC    4    1  0.638941  0.625363  0.599964  0.641299  ...  0.524715   
            2  0.636077  0.697158  0.613994  0.613293  ...  0.524942   

                     22        23        24        25        26        27  \
metric fold                                                                 
BS     0    1  0.058504  0.046140  0.053152  0.061784  0.050324  0.065542   
            2  0.034735  0.019682  0.032066  0.023314  0.031648  0.027103   
AUC    0    1  0.531114  0.499528  0.478013  0.578898  0.500812  0.414534   
            2  0.495373  0.510674  0.517435  0.542218  0.504568  0.725689   
BS     1    1  0.039984  0.049740  0.037045  0.054407  0.063798  0.056508   
            2  0.024147  0.021958  0.020464  0.024052  0.030794  0.022079   
AUC    1    1  0.619649  0.576780  0.408460  0.472122  0.534623  0.390513   
            2  0.564492  0.474570  0.610849  0.5

Results of the AUC(t), BS(t) from the cross-validation procedure to each of the folds and each of the risks:

In [5]:
cross_validator.results

1         2         3         4         5         6   \
metric fold                                                                 
BS     0    1  0.063049  0.053907  0.045755  0.043523  0.042300  0.046062   
            2  0.024376  0.019168  0.019545  0.017727  0.019594  0.017552   
AUC    0    1  0.652021  0.643642  0.641287  0.643345  0.667766  0.643727   
            2  0.672119  0.677260  0.688408  0.646792  0.686449  0.659598   
BS     1    1  0.063104  0.051763  0.047522  0.045185  0.039390  0.040410   
            2  0.026567  0.020499  0.023158  0.019674  0.017881  0.018550   
AUC    1    1  0.637347  0.653555  0.670154  0.653391  0.674059  0.664124   
            2  0.702004  0.665337  0.698247  0.670620  0.612123  0.662859   
BS     2    1  0.064886  0.049643  0.045128  0.044718  0.040647  0.041574   
            2  0.024451  0.016186  0.021169  0.021732  0.021547  0.021889   
AUC    2    1  0.659157  0.654793  0.649652  0.659449  0.674832  0.660598   
            2  0.657810  0.657075  0.706675  0.713073  0.645342  0.666765   
BS     3    1  0.059994  0.050895  0.045448  0.043562  0.037141  0.043832   
            2  0.024225  0.021932  0.019773  0.015853  0.017388  0.017645   
AUC    3    1  0.659007  0.665953  0.682254  0.636794  0.677254  0.626931   
            2  0.702751  0.677411  0.691677  0.699105  0.657049  0.655707   
BS     4    1  0.064885  0.053155  0.047227  0.045384  0.041575  0.035789   
            2  0.023856  0.019616  0.022393  0.021449  0.020102  0.016353   
AUC    4    1  0.659357  0.636783  0.654152  0.637880  0.631202  0.647121   
            2  0.672312  0.657230  0.687828  0.674431  0.653015  0.616802   

                     7         8         9         10  ...        21  \
metric fold                                            ...             
BS     0    1  0.037669  0.039918  0.038884  0.039600  ...  0.036635   
            2  0.021547  0.020059  0.016670  0.019916  ...  0.018437   
AUC    0    1  0.655765  0.667565  0.570066  0.653903  ...  0.514947   
            2  0.629191  0.626242  0.693669  0.635776  ...  0.612318   
BS     1    1  0.040533  0.037199  0.035189  0.039389  ...  0.048439   
            2  0.012963  0.019118  0.018793  0.020397  ...  0.016248   
AUC    1    1  0.626964  0.639071  0.622290  0.624590  ...  0.470615   
            2  0.732118  0.652722  0.610839  0.666786  ...  0.564700   
BS     2    1  0.037698  0.035049  0.041622  0.038163  ...  0.048869   
            2  0.017873  0.020496  0.015362  0.014820  ...  0.018748   
AUC    2    1  0.594342  0.621234  0.648151  0.634931  ...  0.611217   
            2  0.653272  0.674614  0.565575  0.620555  ...  0.552351   
BS     3    1  0.038703  0.043434  0.035425  0.037136  ...  0.039925   
            2  0.017031  0.018022  0.015532  0.019680  ...  0.022589   
AUC    3    1  0.650321  0.615777  0.639428  0.600239  ...  0.464096   
            2  0.675905  0.646852  0.718233  0.665166  ...  0.594124   
BS     4    1  0.038320  0.040585  0.031461  0.032940  ...  0.038013   
            2  0.019711  0.020459  0.021224  0.017895  ...  0.025120   
AUC    4    1  0.638941  0.625363  0.599964  0.641299  ...  0.524715   
            2  0.636077  0.697158  0.613994  0.613293  ...  0.524942   

                     22        23        24        25        26        27  \
metric fold                                                                 
BS     0    1  0.058504  0.046140  0.053152  0.061784  0.050324  0.065542   
            2  0.034735  0.019682  0.032066  0.023314  0.031648  0.027103   
AUC    0    1  0.531114  0.499528  0.478013  0.578898  0.500812  0.414534   
            2  0.495373  0.510674  0.517435  0.542218  0.504568  0.725689   
BS     1    1  0.039984  0.049740  0.037045  0.054407  0.063798  0.056508   
            2  0.024147  0.021958  0.020464  0.024052  0.030794  0.022079   
AUC    1    1  0.619649  0.576780  0.408460  0.472122  0.534623  0.390513   
            2  0.564492  0.474570  0.610849  0.5

with the integrated AUC and BS to each of folds and each of the risks:

In [6]:
pd.DataFrame.from_records(cross_validator.integrated_auc)

,0,1,2,3,4
1,0.629493,0.627465,0.636286,0.634377,0.622095
2,0.651277,0.647620,0.646869,0.655923,0.642630


In [7]:
pd.DataFrame.from_records(cross_validator.integrated_bs)

,0,1,2,3,4
1,0.048382,0.046735,0.046411,0.046305,0.047747
2,0.020957,0.021362,0.021003,0.020547,0.021381


and lastly, the global AUC and global BS to each of the folds:

In [8]:
print(cross_validator.global_auc)

{0: 0.635987860592661, 1: 0.633667449693418, 2: 0.6395581436224567, 3: 0.6408781976004154, 4: 0.6284087721165604}


In [9]:
print(cross_validator.global_bs)

{0: 0.0402049045296208, 1: 0.03892613679526756, 2: 0.03855544852975435, 3: 0.0385327046054388, 4: 0.039640037102306486}


## References

[1] Meir, Tomer\*, Gutman, Rom\*, and Gorfine, Malka, "PyDTS: A Python Package for Discrete-Time Survival Analysis with Competing Risks" (2022)